<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Ew9WrzmmuruC4yL3PUmVtJjswBYgFvqc?usp=sharing)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
*Empowering the Next Generation of AI Innovators

 ### 🚀 Supabase for GenAI Overview 🌟

 Supabase is an open-source backend-as-a-service (BaaS) that provides scalable,
 real-time databases, authentication, storage, and edge functions.
 It is a powerful tool for **Generative AI (GenAI)** applications,
 enabling **embedding storage, similarity search, and retrieval-augmented generation (RAG).**

 ### 🔑 Key Features for GenAI:

 ✅ Vector Search with `pgvector` 🧠🔎  
 ✅ Scalable PostgreSQL Database for AI Content 📊🔐  
 ✅ Real-time Streaming & Webhooks ⏳🔔  
 ✅ Authentication & User Management 🔑👤  


###**Setup and Installation**

In [ ]:
!pip install supabase python-dotenv openai langchain sentence-transformers tiktoken langchain-community

In [ ]:
import os
from supabase import create_client, Client
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings

### **🔑 Setup Supabase and OpenAI Credentials**


In [ ]:
from google.colab import userdata
import os
import numpy as np

SUPABASE_URL =userdata.get('SUPABASE_URL')
SUPABASE_KEY =userdata.get('SUPABASE_KEY')
OPENAI_API_KEY =userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


### **🛠 Create `ai_content` Table in Supabase (Run in SQL Editor)**


In [ ]:
#Paste in Sql Editor in Supabase
'''
CREATE TABLE ai_content (
  id SERIAL PRIMARY KEY,
  created_at TIMESTAMPTZ DEFAULT NOW(),
  content TEXT NOT NULL
);
'''

In [ ]:
'''
-- First enable vector extension if not already enabled
CREATE EXTENSION IF NOT EXISTS vector;

-- Add vector column (example using 1536-dimension OpenAI embeddings)
ALTER TABLE ai_content ADD COLUMN embedding vector(1536);

'''

In [ ]:
'''
-- For user-specific content storage
ALTER TABLE ai_content ADD COLUMN user_id UUID REFERENCES auth.users(id);

'''

In [ ]:
'''
CREATE OR REPLACE FUNCTION match_ai_content(query_embedding vector(1536), match_count int DEFAULT 5)
RETURNS TABLE(content TEXT, similarity FLOAT)
LANGUAGE SQL STABLE AS $$
SELECT content,
       1 - (embedding <=> query_embedding) AS similarity
FROM ai_content
ORDER BY similarity DESC
LIMIT match_count;
$$;

'''

### **💾 Store Text with AI Embeddings in Supabase**


In [ ]:
def store_with_embedding(text, user_id=None):
    """Generate embeddings for the given text and store them in Supabase"""
    embedding = embeddings_model.embed_query(text)  # Generate embedding
    embedding_array = np.array(embedding).tolist()  # Convert to list for storage

    # Insert into the database
    data = {
        "content": text,
        "embedding": embedding_array
    }
    if user_id:
        data["user_id"] = user_id  # Store user-specific content if provided

    response = supabase.table("ai_content").insert(data).execute()
    return response


### **🔍 Search for Similar AI Content in Supabase**


In [ ]:
def search_similar_content(query, top_k=5):
    """Search for similar content using the match_ai_content function"""

    # Generate query embedding
    query_embedding = embeddings_model.embed_query(query)

    # Call the Supabase RPC function
    response = supabase.rpc("match_ai_content", {
        "query_embedding": query_embedding,
        "match_count": top_k
    }).execute()

    return response


### **📥 Store Sample Content and Perform Similarity Search**


In [ ]:
# Store some sample content
store_with_embedding("The quick brown fox jumps over the lazy dog.")
store_with_embedding("Artificial Intelligence is transforming the world.")
store_with_embedding("Machine learning is a subset of AI.")

# Search for similar content
results = search_similar_content("Tell me about AI")
print(results)


data=[{'content': 'Artificial Intelligence is transforming the world.', 'similarity': 0.884156260366599}, {'content': 'Artificial Intelligence is transforming the world.', 'similarity': 0.884156260366599}, {'content': 'Machine learning is a subset of AI.', 'similarity': 0.85831841404198}, {'content': 'The quick brown fox jumps over the lazy dog.', 'similarity': 0.734227440781245}, {'content': 'The quick brown fox jumps over the lazy dog.', 'similarity': 0.734227440781245}] count=None


### **📊 Fetch and Display All Rows from Supabase**


In [ ]:
response = supabase.table("ai_content").select("*").execute()

print(response.data)

[{'id': 1, 'created_at': '2025-02-04T19:48:39.679258+00:00', 'content': 'The quick brown fox jumps over the lazy dog.', 'embedding': '[0.0015723939,0.005963983,-0.015102495,-0.008559333,-0.011583589,0.01556584,-0.021551738,-0.008646993,-0.010944926,-0.022415811,0.02138894,0.017556963,0.021977512,-0.0026141342,0.0007364971,-0.012203468,0.025972284,-0.008465412,0.0136373285,-0.012366264,-0.0134745315,0.023154655,-0.0052908827,-0.0052783596,0.008077205,0.021025779,0.010362616,0.0008742479,0.0028019764,-0.005810579,0.015816296,-0.0021100917,-0.036266025,-0.0012209729,-0.010926142,-0.0141632855,0.0021179183,-0.0065682083,0.024306754,-0.046634905,0.011464623,0.021601828,-0.025696784,-0.005832494,0.002362113,0.013925352,0.010675686,-0.015603407,-0.03215855,0.010224866,0.0012694987,-1.6313881e-05,-0.01015599,-0.008340185,-0.0075512477,-0.011996841,-0.007864318,-0.0030884354,0.014914653,-0.004724226,0.0054098493,0.012635504,-0.0033623716,-0.007175564,-0.00077837025,-0.010788391,0.0071379957,-0.

### **📄 Add a New  Content**


In [ ]:
new_doc = {

    "content": "The AI revolution is transforming industries worldwide.",
}

response = supabase.table("ai_content").insert(new_doc).execute()
print(response)

data=[{'id': 6, 'created_at': '2025-02-04T20:01:43.947818+00:00', 'content': 'The AI revolution is transforming industries worldwide.', 'embedding': None, 'user_id': None}] count=None


### **✏️ Update an Existing Document's Content**


In [ ]:

updated_doc = {
    "content": "Updated content about Artificial Intelligence and its applications."
}

response = supabase.table("ai_content").update(updated_doc).eq("id", 1).execute()  # Assuming id = 1
print(response)


data=[{'id': 1, 'created_at': '2025-02-04T19:48:39.679258+00:00', 'content': 'Updated content about Artificial Intelligence and its applications.', 'embedding': '[0.0015723939,0.005963983,-0.015102495,-0.008559333,-0.011583589,0.01556584,-0.021551738,-0.008646993,-0.010944926,-0.022415811,0.02138894,0.017556963,0.021977512,-0.0026141342,0.0007364971,-0.012203468,0.025972284,-0.008465412,0.0136373285,-0.012366264,-0.0134745315,0.023154655,-0.0052908827,-0.0052783596,0.008077205,0.021025779,0.010362616,0.0008742479,0.0028019764,-0.005810579,0.015816296,-0.0021100917,-0.036266025,-0.0012209729,-0.010926142,-0.0141632855,0.0021179183,-0.0065682083,0.024306754,-0.046634905,0.011464623,0.021601828,-0.025696784,-0.005832494,0.002362113,0.013925352,0.010675686,-0.015603407,-0.03215855,0.010224866,0.0012694987,-1.6313881e-05,-0.01015599,-0.008340185,-0.0075512477,-0.011996841,-0.007864318,-0.0030884354,0.014914653,-0.004724226,0.0054098493,0.012635504,-0.0033623716,-0.007175564,-0.00077837025,-

### **🗑️ Delete a Document by ID**


In [ ]:
response = supabase.table("ai_content").delete().eq("id", 1).execute()
print(response)

data=[{'id': 1, 'created_at': '2025-02-04T19:48:39.679258+00:00', 'content': 'Updated content about Artificial Intelligence and its applications.', 'embedding': '[0.0015723939,0.005963983,-0.015102495,-0.008559333,-0.011583589,0.01556584,-0.021551738,-0.008646993,-0.010944926,-0.022415811,0.02138894,0.017556963,0.021977512,-0.0026141342,0.0007364971,-0.012203468,0.025972284,-0.008465412,0.0136373285,-0.012366264,-0.0134745315,0.023154655,-0.0052908827,-0.0052783596,0.008077205,0.021025779,0.010362616,0.0008742479,0.0028019764,-0.005810579,0.015816296,-0.0021100917,-0.036266025,-0.0012209729,-0.010926142,-0.0141632855,0.0021179183,-0.0065682083,0.024306754,-0.046634905,0.011464623,0.021601828,-0.025696784,-0.005832494,0.002362113,0.013925352,0.010675686,-0.015603407,-0.03215855,0.010224866,0.0012694987,-1.6313881e-05,-0.01015599,-0.008340185,-0.0075512477,-0.011996841,-0.007864318,-0.0030884354,0.014914653,-0.004724226,0.0054098493,0.012635504,-0.0033623716,-0.007175564,-0.00077837025,-